In [1]:
from basic.utils.run.train import train
from basic.utils.common_utils import put_data_to_gpu
from basic.utils.run.train import Trainer
from basic.utils.vis_utils import VisualWindow
import easydict
import yaml
import torch
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

/home/ph/ENV/anaconda3/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


In [2]:
train_cfg_path = '../kitti/cfg/train_cfg.yaml'
with open(train_cfg_path, 'rb') as f:
    train_cfg = easydict.EasyDict(yaml.load(f))
train_cfg

<ipython-input-2-b733ea9d0706>:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  train_cfg = easydict.EasyDict(yaml.load(f))


{'DATASET_CONFIG': '/home/ph/Desktop/PointCloud/utils_my/kitti/cfg/kitti_dataset.yaml',
 'CLASS_NAMES': ['Car'],
 'BATCH': 6,
 'EPOCHS': 5,
 'MODEL': {'NAME': 'SECOND',
  'CONFIG': '/home/ph/Desktop/PointCloud/utils_my/basic/model/model_cfg/second.yaml'},
 'OPTIMIZER': {'NAME': 'Adam', 'LR': 0.003}}

In [3]:
# train(train_cfg)

In [4]:
from basic.model.second import SECOND
from pathlib import Path
from kitti.kitti_dataset import get_dataloader

cfg_path = Path("../kitti/cfg/kitti_dataset.yaml")
dataloader = get_dataloader(data_cfg_path=cfg_path, class_name_list=['Car'], batch_size=6, training=True)
for data in dataloader:
    test_data = data
    break
test_data = put_data_to_gpu(test_data)
test_data

/home/ph/Desktop/PointCloud/utils_my/kitti/kitti_dataset.py:30: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg = EasyDict(yaml.load(f))


{'frame_id': array(['006919', '007213', '007153', '006838', '006500', '007152'],
       dtype='<U6'),
 'gt_boxes': tensor([[[ 1.8864e+01, -6.6366e+00, -7.6978e-01,  4.2167e+00,  1.7660e+00,
            1.8851e+00,  7.8810e-01,  1.0000e+00],
          [ 3.1965e+01, -1.9608e+01, -6.8011e-01,  3.6710e+00,  1.6271e+00,
            1.5081e+00, -2.3219e+00,  1.0000e+00],
          [ 3.9207e+01, -1.2665e+01, -7.3814e-01,  3.6412e+00,  1.6073e+00,
            1.4882e+00, -2.7319e+00,  1.0000e+00],
          [ 5.2057e+00, -5.0620e+00, -7.0900e-01,  3.2940e+00,  1.6172e+00,
            1.5378e+00, -7.6190e-01,  1.0000e+00],
          [ 1.0225e+01, -1.1101e+01, -7.1154e-01,  3.5321e+00,  1.6867e+00,
            1.5875e+00, -6.1190e-01,  1.0000e+00],
          [ 4.1862e+00, -1.7726e+01, -4.7673e-01,  4.2961e+00,  1.6271e+00,
            1.6073e+00, -3.1119e+00,  1.0000e+00],
          [ 3.0324e+00, -1.1087e+01, -8.3326e-01,  3.0856e+00,  1.5875e+00,
            1.5974e+00, -1.4819e+00,  1.0000e+00

In [5]:
%matplotlib auto
from basic.utils.run.train import Predictor
model_path = '/home/ph/Desktop/PointCloud/utils_my/basic/utils/run/check_point/2021-10-22::15:22:49/epoch1.pkl'
predictor = Predictor(model_path, class_names=['Car'], data_cfg_path=cfg_path, batch_size=2)
# predictor.predict_and_visualize()
preds_dict = predictor.predict_bbox(test_data)
preds_dict

Using matplotlib backend: Qt5Agg


{'pred_bbox': None, 'pred_bbox_labels': None, 'frame_inds': None}

In [6]:
from kitti.io.kitti_io import KittiIo
file_reader = KittiIo(root="/home/ph/Dataset/KITTI/training")
if preds_dict['frame_inds'] is not None:
    w = VisualWindow(mode='3d', file_reader=file_reader)
    for i in preds_dict['frame_inds'].unique():
        mask = test_data['points'][:, 0] == i
        w.draw_point_cloud(test_data['points'][mask][:, 1:].cpu())
        w.draw_boxes3d(test_data['gt_boxes'][i].cpu().numpy(), 'corner')
        w.draw_boxes3d(preds_dict['pred_bbox'].cpu().numpy(), 'corner', c='r')

In [7]:
test_data['gt_boxes'].shape

torch.Size([6, 18, 8])